In [2]:
import pandas as pd
import numpy as np
import bokeh.io
import bokeh.plotting
import iqplot
from bokeh.io import curdoc
curdoc().theme = 'dark_minimal'
bokeh.io.output_notebook()


Loading BokehJS ...

In [8]:
file_path = 'Aurora_RNASeq_UQN_Counts_T.csv'
import scipy.stats as ss
from statsmodels.stats.multitest import multipletests

file_path = 'Aurora_RNASeq_UQN_Counts_T.csv'
df = pd.read_csv(file_path)

df = df.set_index('Unnamed: 0')
meta = pd.read_csv('Aurora_Metadata.csv')
meta = meta.set_index('Unnamed: 0')
coding_genes_df = pd.read_csv('coding_genes.csv')
cod_genes = list(np.unique(coding_genes_df['gene_name']))
df = df.loc[df.index.isin(cod_genes)]
counts = df.T
counts['Type'] = list(meta['Type'])
counts['Location'] = list(meta['Location'])
met = counts.loc[counts['Type'] == 'Metastatic']
pri = counts.loc[counts['Type'] == 'Primary']


In [ ]:
tum_counts = counts.loc[counts['Type'] != 'Normal'].drop(columns = ['Location'])
ave_exprs = tum_counts.drop(columns = 'Type').mean(axis=0)
cutoff = np.percentile(ave_exprs, 44.49)

met_counts = tum_counts.loc[tum_counts['Type'] == 'Metastatic']
pri_counts = tum_counts.loc[tum_counts['Type'] != 'Metastatic']


met_counts = met_counts.drop(columns='Type').loc[:, tum_counts.drop(columns = 'Type').mean() > cutoff]
pri_counts = pri_counts.drop(columns='Type').loc[:, tum_counts.drop(columns = 'Type').mean() > cutoff]

log2fc_list = []
pval_list = []
for gene in met_counts.columns:
    met_expr = met_counts[gene]
    pri_expr = pri_counts[gene]
    met_mean = met_expr.mean()
    pri_mean = pri_expr.mean()
    log2fc = np.log2((met_mean + 1e-10) / (pri_mean+1e-10))
    
    stat, pval = ss.ttest_ind(met_expr, pri_expr,  equal_var=False)
    log2fc_list.append(log2fc)
    pval_list.append(pval)

In [ ]:
adj_pvals = ss.false_discovery_control(pval_list, method = 'bh')

my_toptable = pd.DataFrame({
    'Gene': met_counts.columns,
    'log2FC': log2fc_list,
    'p-value':pval_list,
    'adj p-value': adj_pvals
}).set_index('Gene')

my_toptable

In [ ]:
norm = counts.loc[counts['Type'] == 'Normal']

nonsig = my_toptable.loc[(my_toptable['adj p-value'] > 0.05)]
sig = my_toptable.loc[(my_toptable['adj p-value'] < 0.05) ]



In [ ]:
high_met = sig.loc[sig['log2FC'] > 0]
high_pri = sig.loc[sig['log2FC'] < 0]
high_met = high_met.reset_index()
norm = counts.loc[counts['Type'] == 'Normal']
norm_counts = norm.drop(columns = ['Type','Location'])
#norm_counts = norm_counts.drop('AUR-AFE4-NT1-A-1-1-R-A742-41')
norm_counts.loc['Mean'] = norm_counts.mean()
norm_counts.loc['Max'] = norm_counts.max()
norm_counts = norm_counts.T
try:
    high_met = high_met.set_index('X', drop=True)
except:
    pass
try: 
    high_pri = high_pri.set_index('X', drop=True)
except:
    pass

high_met = high_met.set_index('Gene')
high_met_n = norm_counts.loc[high_met.index.tolist()]
high_met['Norm Mean'] = high_met_n['Mean']
high_met['Norm Max'] = high_met_n['Max']
high_met['True Avg'] = met[high_met.index].mean(axis=0)
high_pri['True Avg'] = pri[high_pri.index].mean(axis=0)
high_pri_n = norm_counts.loc[high_pri.index.tolist()]
high_pri['Norm Mean'] = high_pri_n['Mean']
high_pri['Norm Max'] = high_pri_n['Max']
safe_high_met = high_met.loc[high_met['Norm Max'] < 0.5*high_met['True Avg']]
safe_high_pri = high_pri.loc[high_pri['Norm Max'] < 0.5*high_pri['True Avg']]